<a href="https://colab.research.google.com/github/saiteja-d/spark_course/blob/master/Working_with_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with columns

## Download and install Spark

In [0]:
!ls

In [0]:
#!apt-get update
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#!pip install -q findspark

## Setup environment

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [0]:
#!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
#!ls -l

In [0]:
#!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
#!ls -l

In [3]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

+-------+-----------+-------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|               Date|               Block|IUCR| Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|5705297|   HN501051|2007-07-31 19:01:52|  038XX S KEDZIE AVE|0460|      BATTERY|              SIMPLE|              STREET| false|   false|0913|     00

## Working with columns

**Display only the first 5 rows of the column name IUCR **

In [0]:
ls


reported-crimes.csv  spark-2.3.1-bin-hadoop2.7/     spark-warehouse/
sample_data/         spark-2.3.1-bin-hadoop2.7.tgz


In [4]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|0460|
|2825|
|0810|
|2820|
|0460|
+----+
only showing top 5 rows



In [6]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|0460|
|2825|
|0810|
|2820|
|0460|
+----+
only showing top 5 rows



  **Display only the first 4 rows of the column names Case Number, Date and Arrest**

In [7]:
rc.select('case Number', 'Date', 'Arrest').show(5)

+-----------+-------------------+------+
|case Number|               Date|Arrest|
+-----------+-------------------+------+
|   HN501051|2007-07-31 19:01:52| false|
|   HN514327|2007-08-06 18:00:00| false|
|   HN508549|2007-08-03 20:00:00| false|
|   HN514231|2007-08-03 15:00:00| false|
|   HN507817|2007-08-04 01:34:58|  true|
+-----------+-------------------+------+
only showing top 5 rows



** Add a column with name One, with entries all 1s **

In [0]:
from pyspark.sql.functions import lit

In [9]:
rc.withColumn('One', lit(1)).show(5)

+-------+-----------+-------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|     ID|Case Number|               Date|               Block|IUCR| Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|One|
+-------+-----------+-------------------+--------------------+----+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|5705297|   HN501051|2007-07-31 19:01:52|  038XX S KEDZIE AVE|0460|      BATTERY|              SIMPLE|              STREET| false|   false|

** Remove the column IUCR **

In [0]:
rc = rc.drop('IUCR')


In [14]:
rc.show(5)

+-------+-----------+-------------------+--------------------+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|               Date|               Block| Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+-------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|5705297|   HN501051|2007-07-31 19:01:52|  038XX S KEDZIE AVE|      BATTERY|              SIMPLE|              STREET| false|   false|0913|     009|  12|            5